In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 14.8 MB/s eta 0:00:00


In [ ]:
import gdown
import os
import yaml
import shutil
import torch
#import random
import numpy as np
from ultralytics import YOLO
#from sklearn.model_selection import train_test_split
# методы для отрисовки изображений
from PIL import Image

In [ ]:
pip install --upgrade ultralytics

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import locale
#locale.getpreferredencoding = lambda: 'UTF-8'

In [ ]:
#Скачиваем видео и  веса
!gdown 1ErRtAyOsaY18gIOlsQgl7WO0GPQ6_3nw
!gdown 1zDbfknsUz9fgnQkEvSWd5eFU_tWuyUfx

Downloading...
From: https://drive.google.com/uc?id=1ErRtAyOsaY18gIOlsQgl7WO0GPQ6_3nw
To: /content/VOP.mp4
100% 96.7M/96.7M [00:02<00:00, 45.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zDbfknsUz9fgnQkEvSWd5eFU_tWuyUfx
To: /content/yolo8n_best27.pt
100% 6.25M/6.25M [00:00<00:00, 28.5MB/s]


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
from google.colab.patches import cv2_imshow

# Загружаем модель YOLO v8
model = YOLO('yolo8n_best27.pt')  # Используй нужную модель

# Открываем видеофайл
video_path = 'VOP.mp4'  # Путь к исходному видеофайлу
cap = cv2.VideoCapture(video_path)

# Получаем параметры исходного видео
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Размер блоков
block_size = 640

# Создаем VideoWriter для сохранения нового видео
out = cv2.VideoWriter('output_with_predictions.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_count = 0  # Счётчик кадров

while cap.isOpened():
    # Чтение кадра
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1

    # Обрабатываем каждый 10-й кадр
    if frame_count % 10 == 0:
        height, width, _ = frame.shape  # Определяем размеры кадра

        # Определяем количество полных блоков по горизонтали и вертикали
        num_blocks_x = width // block_size
        num_blocks_y = height // block_size

        # Остатки
        remainder_x = width % block_size
        remainder_y = height % block_size

        processed_blocks = []  # Список для хранения обработанных блоков

        # Проходим по полным блокам
        for i in range(num_blocks_y):
            for j in range(num_blocks_x):
                x = j * block_size
                y = i * block_size

                # Вырезаем блок 640x640
                block = frame[y:y+block_size, x:x+block_size]

                # Прогоняем блок через YOLO для предсказания
                results = model.predict(block, verbose=False)

                # Визуализируем результат
                block_with_predictions = results[0].plot()

                # Добавляем обработанный блок в список
                processed_blocks.append((block_with_predictions, x, y))

        # Обрабатываем остаточные блоки по ширине
        if remainder_x > 0:
            for i in range(num_blocks_y):
                x = num_blocks_x * block_size
                y = i * block_size

                # Вырезаем остаточную область по ширине
                block = frame[y:y+block_size, x:width]

                # Заполняем до 640x640 черным
                #padded_block = np.zeros((block_size, block_size, 3), dtype=np.uint8)
                #padded_block[:block.shape[0], :block.shape[1]] = block

                # Прогоняем через YOLO
                results = model.predict(block, verbose=False)
                block_with_predictions = results[0].plot()

                # Добавляем в список
                processed_blocks.append((block_with_predictions, x, y))

        # Обрабатываем остаточные блоки по высоте
        if remainder_y > 0:
            for j in range(num_blocks_x):
                x = j * block_size
                y = num_blocks_y * block_size

                # Вырезаем остаточную область по высоте
                block = frame[y:height, x:x+block_size]
                # Заполняем до 640x640 черным

                #padded_block = np.zeros((block_size, block_size, 3), dtype=np.uint8)
                #padded_block[:block.shape[0], :block.shape[1]] = block


                # Прогоняем через YOLO
                results = model.predict(block, verbose=False)
                block_with_predictions = results[0].plot()

                # Добавляем в список
                processed_blocks.append((block_with_predictions, x, y))

        # Обрабатываем угол (остаток по ширине и высоте)
        if remainder_x > 0 and remainder_y > 0:
            x = num_blocks_x * block_size
            y = num_blocks_y * block_size

            # Вырезаем остаточную область по ширине и высоте
            block = frame[y:height, x:width]

            # Заполняем до 640x640 черным
            #padded_block = np.zeros((block_size, block_size, 3), dtype=np.uint8)
            #padded_block[:block.shape[0], :block.shape[1]] = block

            # Прогоняем через YOLO
            results = model.predict(block, verbose=False)
            block_with_predictions = results[0].plot()

            # Добавляем в список
            processed_blocks.append((block_with_predictions, x, y))

        # Собираем обратно весь кадр
        output_frame = np.zeros_like(frame)

        for processed_block, x, y in processed_blocks:
            h, w, _ = processed_block.shape
            output_frame[y:y+h, x:x+w] = processed_block

        # Отображаем итоговый кадр с результатами предсказаний
        cv2_imshow(output_frame)

        # Записываем итоговый кадр в новый видеофайл
        out.write(output_frame)

# Освобождаем ресурсы
cap.release()
out.release()


Output hidden; open in https://colab.research.google.com to view.

[Файл output каждый 10 кадр](https://drive.google.com/file/d/1BS_RG62Rc93_jDnly9o71EKzbLynOm8R/view?usp=sharing)

# Многопоточность

In [6]:
import cv2
from ultralytics import YOLO
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Загружаем модель YOLO
model = YOLO('yolo8n_best27.pt')

# Открываем видеофайл
video_path = 'VOP.mp4'
cap = cv2.VideoCapture(video_path)

# Получаем параметры исходного видео
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Размер блоков
block_size = 640

# Создаем VideoWriter для сохранения нового видео
out = cv2.VideoWriter('output_with_predictions1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_count = 0
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Функция для параллельной обработки блока
def process_block(block):
    results = model.predict(block, verbose=False)
    return results[0].plot()

with tqdm(total=total_frames) as pbar:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Обрабатываем каждый 10-й кадр
        if frame_count % 10 == 0:
            height, width, _ = frame.shape
            num_blocks_x = width // block_size
            num_blocks_y = height // block_size
            remainder_x = width % block_size
            remainder_y = height % block_size

            blocks = []

            # Разбиваем кадр на блоки
            for i in range(num_blocks_y):
                for j in range(num_blocks_x):
                    x = j * block_size
                    y = i * block_size
                    block = frame[y:y + block_size, x:x + block_size]
                    blocks.append((block, (y, x)))

            if remainder_x > 0:
                for i in range(num_blocks_y):
                    x = num_blocks_x * block_size
                    y = i * block_size
                    block = frame[y:y + block_size, x:width]
                    blocks.append((block, (y, x)))

            if remainder_y > 0:
                for j in range(num_blocks_x):
                    x = j * block_size
                    y = num_blocks_y * block_size
                    block = frame[y:height, x:x + block_size]
                    blocks.append((block, (y, x)))

            if remainder_x > 0 and remainder_y > 0:
                x = num_blocks_x * block_size
                y = num_blocks_y * block_size
                block = frame[y:height, x:width]
                blocks.append((block, (y, x)))

            # Параллельная обработка блоков
            with ThreadPoolExecutor() as executor:
                futures = {executor.submit(process_block, block): idx for idx, (block, coords) in enumerate(blocks)}
                processed_blocks = [None] * len(blocks)

                for future in as_completed(futures):
                    index = futures[future]  # Получаем индекс блока
                    processed_block = future.result()
                    processed_blocks[index] = processed_block

            # Создаем пустой кадр для выходного видео
            output_frame = np.zeros_like(frame)

            # Вставляем обработанные блоки на соответствующие места
            for (block, (y, x)), processed_block in zip(blocks, processed_blocks):
                if processed_block is not None:
                    h, w, _ = processed_block.shape
                    output_frame[y:y + h, x:x + w] = processed_block

            # Записываем обработанный кадр в видео
            out.write(output_frame)
        else:
            # Если кадр не обрабатывается, просто записываем его как есть
            out.write(frame)

        pbar.update(1)

# Освобождаем ресурсы
cap.release()
out.release()




  0%|          | 4/1661 [00:00<00:45, 36.31it/s]

Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 1661/1661 [02:09<00:00, 12.86it/s]


[Файл output все кадры](https://drive.google.com/file/d/1_XyaiizNI6FMJhbldiK0Ijj78X55PTXj/view?usp=sharing)